## Term Frequency-Inverse Document Frequency (TF-IDF)

TF = count of terms / total count of terms
IDF = log (total # of documents / number of documents t is present in)

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
import spacy

In [66]:
df = pd.read_csv("train.csv")
nlp = spacy.load("en_core_web_sm")
df = df.fillna("")

In [67]:
X = df[['keyword', 'location', 'text']]
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [84]:
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin

class textPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.text_columns = text_columns
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        X_transformed = X.copy()
        
        # Apply the text processing function to each specified text column
        for col in self.text_columns:
            X_transformed[col] = X_transformed[col].apply(self.process_text)
        
        return X_transformed
    def process_text(self, text):
        doc = nlp(text)  # Process the string with the NLP model
        no_stop_words = [token.text for token in doc if not token.is_stop]
        return " ".join(no_stop_words)


text_transformer = Pipeline(steps=[
    ('preprocessor', textPreprocessor()),
    ('tfidf', TfidfVectorizer())
])

text_columns = ['text', 'location', 'keyword']
preprocessor = ColumnTransformer(
    transformers=[
        ('Text', text_transformer, text_columns)
    ])

clf = Pipeline([
    ('txt preprocessor', preprocessor),
    ('rf', RandomForestClassifier())
])

In [70]:
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)  # Should be (n_samples,)

X_train shape: (5329, 3)
y_train shape: (5329,)


In [85]:
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

ValueError: Found input variables with inconsistent numbers of samples: [3, 5329]